<a href="https://de.linkedin.com/in/delafuenteaguilerapablo/de-de"><img src="https://media-exp1.licdn.com/dms/image/C4D03AQElWBXe___WeQ/profile-displayphoto-shrink_200_200/0?e=1609372800&v=beta&t=IdUiY7Y78vTI3jcLbSWUboqDUTKDU2XgKg4KPptmGVY" width="200" align="right"></a>

# Capstone Project - The Battle of Neighborhoods (Week 2)

## Project from Pablo de la Fuente Aguilera
### I´m an engineer and you can see my full background on my Linkedin profile: https://de.linkedin.com/in/delafuenteaguilerapablo/
### This Notebook is part of the Capstone Project.
### This Project is the last Assignment of the IBM Professional Certificate in Data Science, offered by Coursera.org

Importing Libraries

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import urllib.request
import json, time, os
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
import folium
from area import area
from termcolor import colored

print('Libraries successfully imported.')

Libraries successfully imported.


## We present the Problem

We can suppose that we are thinking about opening an Mechanical Repairing Center in Madrid. This could be a good business opportunity but we need to carry out a market research in order to establish a long-term success.

To start with, we will analyse the existing businesses of this category in Madrid. And we will sort them by neighborhood, in order to identify the best possible location.

## Downloading and loading the data about the city of Madrid



In [3]:
# The data was downloaded with the name madrid_data.json
# wget -q -O 'barrios_municipio_madrid.json' https://datos.comunidad.madrid/catalogo/dataset/91857345-056b-4aa7-96f5-6310dac619bf/resource/b9cf6d43-79b0-400e-82de-ca00de21f6bb/download/barrios_municipio_madrid.jsonfixed_ESP_MAD_adm6.json
print('Data downloaded and loaded!')

with open('madrid_data.json') as json_data:
    madrid_data = json.load(json_data)

Data downloaded and loaded!


We create a Dataframe with the downloaded data

In [4]:
neighborhoods_data = madrid_data['data']
# We define the columns of the dataframe
column_names = ['District', 'Neighborhood', 'Latitude', 'Longitude', 'Surface (km2)', 'Density (hab/km2)'] 

# We start and initialize the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods['District'] = neighborhoods['District'].astype('str') 
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].astype('str') 

for data in neighborhoods_data:
    borough = data['distrito_nombre']
    neighborhood_name = data['barrio_nombre']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = float(neighborhood_latlon[1])
    neighborhood_lon = float(neighborhood_latlon[0])

    surface = data['superficie_km2'] 
    density = data['densidad_por_km2']
    price = data['precio_m2']
    
    neighborhoods = neighborhoods.append({'District': borough, 'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat, 'Longitude': neighborhood_lon, 
                                          'Surface (km2)': surface, 'Density (hab/km2)': density, 'Price (€/m2)': price}, ignore_index=True)
barrios = neighborhoods

# We show the Dataframe
neighborhoods.head()

,District,Neighborhood,Latitude,Longitude,Surface (km2),Density (hab/km2),Price (€/m2)
0,Centro,Palacio,-3.713134,40.415325,1.46,15323.287671,4852.0
1,Centro,Embajadores,-3.702543,40.409444,1.03,43345.631068,4479.0
2,Centro,Cortes,-3.696785,40.415439,0.59,17850.847458,5272.0
3,Centro,Justicia,-3.695976,40.423497,0.75,21866.666667,5893.0
4,Centro,Universidad,-3.706963,40.426121,0.93,33051.612903,5282.0


In [5]:
print("We have", neighborhoods.shape[0], " different neighborhoods in total")

We have 128  different neighborhoods in total


In [6]:
# We take the Neighborhood Sol as the middle point in the city and store their coordinates
latitude_sol = float(neighborhoods[neighborhoods['Neighborhood'] == 'Sol']['Latitude'])
longitude_sol = float(neighborhoods[neighborhoods['Neighborhood'] == 'Sol']['Longitude'])
defaultLatLong = [longitude_sol, latitude_sol]
print("The coordinates of our reference point are", latitude_sol, ",", longitude_sol)

#neighborhoods['District'].dtype

The coordinates of our reference point are -3.704406 , 40.417031


We show our neighborhoods in a map using Folium

In [7]:
locations1 = neighborhoods[['Longitude', 'Latitude']]
locationlist1 = locations1.values.tolist()

map_madrid1 = folium.Map(location=[longitude_sol, latitude_sol], zoom_start=12)
for point in range(0, neighborhoods.shape[0]):
    folium.Marker(locationlist1[point], popup= neighborhoods['Neighborhood'][point]).add_to(map_madrid1)
    
map_madrid1

## We analyze the possible neighborhoods in Madrid based on the price of the square meters

We don´t want to spend more than 1.000.000 € on the adquisition of the retail space and our desire is to have a retail store of around 300 square meters. So we´ll stablish a filter between those places which have a Price (€/m2) between 2500 and 3500 

In [8]:
low_price = 2600
high_price = 4200

barrios_data1 = neighborhoods[neighborhoods['Price (€/m2)'] < high_price ].reset_index(drop=True)
barrios_data1 = barrios_data1[barrios_data1['Price (€/m2)'] > low_price ].reset_index(drop=True)
barrios_data1.head(100)

,District,Neighborhood,Latitude,Longitude,Surface (km2),Density (hab/km2),Price (€/m2)
0,Arganzuela,Imperial,-3.718656,40.405161,0.98,23105.102041,3922.0
1,Arganzuela,Las Acacias,-3.706734,40.401523,1.10,33318.181818,4188.0
2,Arganzuela,La Chopera,-3.699556,40.394639,0.56,35276.785714,3625.0
3,Arganzuela,Delicias,-3.693955,40.398093,1.07,25485.046729,3904.0
4,Arganzuela,Palos de Moguer,-3.694600,40.403759,0.65,39372.307692,3943.0
5,Retiro,PacÃ­fico,-3.677976,40.404721,0.76,44271.052632,4149.0
6,Retiro,Estrella,-3.665524,40.413454,1.02,22701.960784,4155.0
7,Tetuan,Bellas Vistas,-3.707527,40.453071,0.73,38923.287671,3365.0
8,Tetuan,Almenara,-3.695287,40.468889,0.99,22232.323232,3419.0
9,Tetuan,Valdeacederas,-3.702735,40.466435,1.17,21574.358974,3099.0


In [9]:
print("After this filter, we have", barrios_data1.shape[0], " different neighborhoods in total")

After this filter, we have 44  different neighborhoods in total


We create a map using Folium to see which neighborhoods are meeting the price requirements

In [10]:
import folium
# create map of Madrid using latitude and longitude values
map_madrid2 = folium.Map(location=[longitude_sol, latitude_sol], zoom_start=12)

# add markers to map
for lng, lat, district, neighborhood in zip(barrios_data1['Latitude'], barrios_data1['Longitude'], barrios_data1['District'], barrios_data1['Neighborhood']):
    label = '{}, {}'.format(district,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid2)  
    
map_madrid2

## We cluster the Neighborhoods

We cluster them into 4 different groups

In [13]:
# set number of clusters
kclusters = 4

dfBarrios_clustering = barrios_data1.drop(['Neighborhood', 'District'], axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfBarrios_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 1, 3, 1, 1, 3, 1, 3, 3])

In [14]:
# add clustering labels
barrios_data1.insert(0, 'Cluster Labels', kmeans.labels_)


barrios_data1.head() # check the Cluster Label columns!

,Cluster Labels,District,Neighborhood,Latitude,Longitude,Surface (km2),Density (hab/km2),Price (€/m2)
0,3,Arganzuela,Imperial,-3.718656,40.405161,0.98,23105.102041,3922.0
1,1,Arganzuela,Las Acacias,-3.706734,40.401523,1.10,33318.181818,4188.0
2,1,Arganzuela,La Chopera,-3.699556,40.394639,0.56,35276.785714,3625.0
3,3,Arganzuela,Delicias,-3.693955,40.398093,1.07,25485.046729,3904.0
4,1,Arganzuela,Palos de Moguer,-3.694600,40.403759,0.65,39372.307692,3943.0


### Cluster Nr. 0

In [19]:
barrios_cluster_0=barrios_data1.loc[barrios_data1['Cluster Labels'] == 0]
barrios_cluster_0.head(barrios_cluster_0.shape[0])

,Cluster Labels,District,Neighborhood,Latitude,Longitude,Surface (km2),Density (hab/km2),Price (€/m2)
12,0,Fuencarral-El Pardo,PeÃ±a Grande,-3.726833,40.479017,2.86,15476.923077,3283.0
14,0,Fuencarral-El Pardo,La Paz,-3.696045,40.483378,2.17,15578.801843,3580.0
22,0,Moratalaz,Marroquina,-3.647142,40.411364,1.75,15633.714286,2868.0
23,0,Moratalaz,Media Legua,-3.660231,40.408978,1.02,17466.666667,2664.0
24,0,Ciudad Lineal,Ventas,-3.647657,40.426883,3.22,14700.931677,2665.0
27,0,Ciudad Lineal,San Pascual,-3.654677,40.441785,1.06,17079.245283,3406.0
30,0,Hortaleza,Canillas,-3.643754,40.464324,2.56,15820.312500,3241.0
32,0,Hortaleza,Apostol Santiago,-3.659881,40.478759,1.20,12627.500000,2850.0
35,0,Villa de Vallecas,Santa Eugenia,-3.606203,40.382919,2.04,11822.058824,2695.0
39,0,Barajas,Alameda de Osuna,-3.592139,40.455778,1.98,9795.454545,3273.0


### Cluster Nr. 1

In [20]:
barrios_cluster_1=barrios_data1.loc[barrios_data1['Cluster Labels'] == 1]
barrios_cluster_1.head(barrios_cluster_1.shape[0])

,Cluster Labels,District,Neighborhood,Latitude,Longitude,Surface (km2),Density (hab/km2),Price (€/m2)
1,1,Arganzuela,Las Acacias,-3.706734,40.401523,1.10,33318.181818,4188.0
2,1,Arganzuela,La Chopera,-3.699556,40.394639,0.56,35276.785714,3625.0
4,1,Arganzuela,Palos de Moguer,-3.694600,40.403759,0.65,39372.307692,3943.0
5,1,Retiro,PacÃ­fico,-3.677976,40.404721,0.76,44271.052632,4149.0
7,1,Tetuan,Bellas Vistas,-3.707527,40.453071,0.73,38923.287671,3365.0
10,1,Tetuan,Berruguete,-3.704891,40.459591,0.60,40533.333333,3263.0
13,1,Fuencarral-El Pardo,El Pilar,-3.708572,40.475544,1.37,33451.824818,3303.0
21,1,Carabanchel,Comillas,-3.711391,40.392854,0.67,33000.000000,2702.0
25,1,Ciudad Lineal,Quintana,-3.648602,40.435811,0.71,34056.338028,2828.0


### Cluster Nr. 2

In [21]:
barrios_cluster_2=barrios_data1.loc[barrios_data1['Cluster Labels'] == 2]
barrios_cluster_2.head(barrios_cluster_2.shape[0])

,Cluster Labels,District,Neighborhood,Latitude,Longitude,Surface (km2),Density (hab/km2),Price (€/m2)
11,2,Fuencarral-El Pardo,Fuentelarreina,-3.743380,40.477849,1.51,2113.907285,3439.0
15,2,Fuencarral-El Pardo,Valverde,-3.682246,40.498379,9.03,6642.857143,3462.0
16,2,Fuencarral-El Pardo,Mirasierra,-3.707996,40.491460,6.91,4465.701881,3645.0
17,2,Moncloa-Aravaca,Ciudad Universitaria,-3.730594,40.443199,14.14,1135.007072,3652.0
19,2,Moncloa-Aravaca,El PlantÃ­o,-3.822546,40.471137,3.56,764.887640,3742.0
20,2,Moncloa-Aravaca,Aravaca,-3.784802,40.457048,5.86,4384.300341,3742.0
28,2,Ciudad Lineal,Atalaya,-3.665093,40.463866,0.25,6240.000000,3020.0
29,2,Hortaleza,Palomas,-3.614780,40.452846,1.14,5859.649123,3732.0
33,2,Hortaleza,Valdefuentes,-3.644558,40.495833,15.19,3400.526662,3813.0
36,2,San Blas-Canillejas,Rosas,-3.604156,40.428114,9.26,3403.023758,3121.0


### Cluster Nr. 3

In [22]:
barrios_cluster_3=barrios_data1.loc[barrios_data1['Cluster Labels'] == 3]
barrios_cluster_3.head(barrios_cluster_3.shape[0])

,Cluster Labels,District,Neighborhood,Latitude,Longitude,Surface (km2),Density (hab/km2),Price (€/m2)
0,3,Arganzuela,Imperial,-3.718656,40.405161,0.98,23105.102041,3922.0
3,3,Arganzuela,Delicias,-3.693955,40.398093,1.07,25485.046729,3904.0
6,3,Retiro,Estrella,-3.665524,40.413454,1.02,22701.960784,4155.0
8,3,Tetuan,Almenara,-3.695287,40.468889,0.99,22232.323232,3419.0
9,3,Tetuan,Valdeacederas,-3.702735,40.466435,1.17,21574.358974,3099.0
18,3,Moncloa-Aravaca,Valdezarza,-3.715617,40.464359,1.44,20317.361111,2989.0
26,3,Ciudad Lineal,ConcepciÃ³n,-3.647228,40.441070,0.90,22828.888889,3259.0
31,3,Hortaleza,Pinar del Rey,-3.646148,40.474450,2.64,19682.954545,2887.0
34,3,Villaverde,Los Rosales,-3.688816,40.354694,1.51,24163.576159,2869.0


We represent the clusters in a map

In [46]:
map_clusters = folium.Map(location=[longitude_sol, latitude_sol], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(barrios_data1['Longitude'], barrios_data1['Latitude'], barrios_data1['Neighborhood'], barrios_data1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

## We start taking Venues of the Neighborhood using Foursquare

### First we define the category filter

In [191]:
LIMIT = 200 
radius = 500 
FOURSQUARE_CLIENT_ID = 'IC1ASQFZMVEWFJTS5JWKEJQRWIRK040ADJNN4JEPEUNLNLTB' # The Foursquare ID
FOURSQUARE_CLIENT_SECRET = '5I3L25G4TIQ12JSPEOYHK3JH1EEDC5KKE0EHQICAWIOA0AUK' # The Foursquare Secret
VERSION = '20200412'

We create the functions to extract the venues using the Foursquare API

In [192]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getDfFoursquareCategories():
    url = 'https://api.foursquare.com/v2/venues/categories'
    params = dict(
        client_id=os.environ.get('FOURSQUARE_CLIENT_ID'),
        client_secret=os.environ.get('FOURSQUARE_CLIENT_SECRET'),
        v='20180605'
    )
    
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    data_categories = data['response']['categories']
    dfcategoriesRaw = json_normalize(data_categories) # flatten JSON
    dfCategories = json_normalize(dfcategoriesRaw['categories'].explode()).drop(['pluralName','shortName','categories','icon.prefix','icon.suffix'], axis = 1)
    return dfCategories
    
def getDfFoursquareNearbyVenues(lat = defaultLatLong[0], lng = defaultLatLong[1], category = '4d4b7105d754a06372d81259' ,limit = 10, radius = 500):
    url = 'https://api.foursquare.com/v2/venues/explore'
    params = dict(
        client_id=os.environ.get('FOURSQUARE_CLIENT_ID'),
        client_secret=os.environ.get('FOURSQUARE_CLIENT_SECRET'),
        v='20180605',
        ll='%s,%s' % (lat, lng),
        radius='%s' % (radius),
        categoryId='%s' % (category),
        limit=limit
    )
    
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    
    venues = data['response']['groups'][0]['items']
    nearby_venues = json_normalize(venues) # flatten JSON
    
    # filter columns 
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    if (len(nearby_venues.index) > 0): 
        nearby_venues = nearby_venues.loc[:, filtered_columns]
        # filter the category for each row
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
        # clean columns
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        
    return nearby_venues

def getDfFoursquareByNeighborhood(neighborhoodName, neighborhoodLat, neighborhoodLng, category = '4d4b7105d754a06372d81259', radius = 500):
    nearby_venues = getDfFoursquareNearbyVenues(lat = neighborhoodLat, lng = neighborhoodLng, category = category , radius = radius ); # A Panda Dataframe
    venues = nearby_venues.rename(columns={'name': 'Venue','lat': 'Venue Latitude', 'lng': 'Venue Longitude', 'categories': 'Venue Category'})
    venues['Neighborhood'] = neighborhoodName
    venues['Neighborhood Latitude'] = neighborhoodLat
    venues['Neighborhood Longitude'] = neighborhoodLng    
    return venues

def getNearbyVenues(names, latitudes, longitudes, categories = [],radius=7500):
    nearbyVenues = pd.DataFrame([], columns = ['Venue','Venue Category','Venue Latitude','Venue Longitude','Neighborhood','Neighborhood Latitude','Neighborhood Longitude'])
    for name, lat, lng in zip(names, latitudes, longitudes):
        for category in categories:
            items = getDfFoursquareByNeighborhood(name, lat, lng, category, radius)
            nearbyVenues = nearbyVenues.append(items, ignore_index=True, sort=False)
            
    print('Total venues found: %s' % len(nearbyVenues.index))     
    return nearbyVenues

def getNearbyVenues2(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(FOURSQUARE_CLIENT_ID, FOURSQUARE_CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)
       

We take the Foursquare Categories

Foursquare categories: https://developer.foursquare.com/docs/build-with-foursquare/categories/

In [193]:
#dfCategoriesRaw = getDfFoursquareCategories()

#Since this is a premium call, we did once and exported the dataframe to a csv file
dfCategoriesRaw = pd.read_csv("categories_foursquare.csv",sep=";")
dfCategoriesRaw.head()

,name,id
0,Arts & Entertainment,4d4b7104d754a06370d81259
1,Amphitheater,56aa371be4b08b9a8d5734db
2,Aquarium,4fceea171983d5d06c3e9823
3,Arcade,4bf58dd8d48988d1e1931735
4,Art Gallery,4bf58dd8d48988d1e2931735


In [194]:
print('We have in total ', dfCategoriesRaw.shape[0],' different categories')

We have in total  941  different categories


Now, we filter the categories using key words

In [195]:
dfCategoriesFilter = dfCategoriesRaw[dfCategoriesRaw['name'].str.contains('(Hotel|Rental|Car Rent|Taxi|Uber|Airbnb|Driver)', regex = True, case=False)].sort_values('name').reset_index(drop = True)
dfCategoriesFilter.head(1000)

,name,id
0,Bike Rental / Bike Share,4e4c9077bd41f78e849722f9
1,Boat Rental,5744ccdfe4b0c0459246b4c1
2,Hotel,4bf58dd8d48988d1fa931735
3,Hotel Bar,4bf58dd8d48988d1d5941735
4,Hotel Pool,4bf58dd8d48988d132951735
5,Rental Car Location,4bf58dd8d48988d1ef941735
6,Rental Service,56aa371be4b08b9a8d573552
7,Taxi,4bf58dd8d48988d130951735
8,Taxi Stand,53fca564498e1a175f32528b
9,Vacation Rental,56aa371be4b08b9a8d5734e1


We eliminate confusing categories

In [196]:
dfCategories = dfCategoriesFilter[dfCategoriesFilter['name'].str.contains('^((?!(Boat|Pool|Vacation|Bar)).)*$', regex = True, case=False)].sort_values('name').reset_index(drop = True)
dfCategories.head(100)

,name,id
0,Bike Rental / Bike Share,4e4c9077bd41f78e849722f9
1,Hotel,4bf58dd8d48988d1fa931735
2,Rental Car Location,4bf58dd8d48988d1ef941735
3,Rental Service,56aa371be4b08b9a8d573552
4,Taxi,4bf58dd8d48988d130951735
5,Taxi Stand,53fca564498e1a175f32528b


In [197]:
print('Now we have only ', dfCategories.shape[0],' different categories')

Now we have only  6  different categories


Finally we have a reduced number of categories. And they have these categories id´s:

In [198]:
dfCategories['id'].to_list()

['4e4c9077bd41f78e849722f9',
 '4bf58dd8d48988d1fa931735',
 '4bf58dd8d48988d1ef941735',
 '56aa371be4b08b9a8d573552',
 '4bf58dd8d48988d130951735',
 '53fca564498e1a175f32528b']

### Now we take all the Venues in the filtered Neighborhoods

In [199]:
#dfVenues = getNearbyVenues(barrios_data1['Neighborhood'],barrios_data1['Latitude'], barrios_data1['Longitude'], dfCategories['id'].to_list())
dfVenues = getNearbyVenues2(names=barrios_data1['Neighborhood'], latitudes=barrios_data1['Latitude'], longitudes=barrios_data1['Longitude'], radius=1000, categoryIds='')
dfVenues.head(200)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Imperial,-3.718656,40.405161,Europcar,-3.746100,40.327000,Rental Car Location
1,Imperial,-3.718656,40.405161,Europcar,-3.789800,40.464700,Rental Car Location
2,Las Acacias,-3.706734,40.401523,Europcar,-3.746100,40.327000,Rental Car Location
3,Las Acacias,-3.706734,40.401523,Europcar,-3.789800,40.464700,Rental Car Location
4,La Chopera,-3.699556,40.394639,Holiday Inn Express,-3.692243,40.273166,Hotel
5,La Chopera,-3.699556,40.394639,Europcar,-3.789800,40.464700,Rental Car Location
6,La Chopera,-3.699556,40.394639,Europcar,-3.746100,40.327000,Rental Car Location
7,Delicias,-3.693955,40.398093,Europcar,-3.746100,40.327000,Rental Car Location
8,Delicias,-3.693955,40.398093,Holiday Inn Express,-3.692243,40.273166,Hotel
9,Delicias,-3.693955,40.398093,Europcar,-3.789800,40.464700,Rental Car Location


In [200]:
print('There are {} uniques categories.'.format(len(dfVenues['Venue Category'].unique())))

There are 2 uniques categories.


In [201]:
# one hot encoding
dfVenues_onehot = pd.get_dummies(dfVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dfVenues_onehot['Neighborhood'] = dfVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfVenues_onehot.columns[-1]] + list(dfVenues_onehot.columns[:-1])
dfVenues_onehot = dfVenues_onehot[fixed_columns]

dfVenues_onehot.head(50)

,Neighborhood,Hotel,Rental Car Location
0,Imperial,0,1
1,Imperial,0,1
2,Las Acacias,0,1
3,Las Acacias,0,1
4,La Chopera,1,0
5,La Chopera,0,1
6,La Chopera,0,1
7,Delicias,0,1
8,Delicias,1,0
9,Delicias,0,1


In [202]:
dfVenues_onehot.shape

(65, 3)

We combine the duplicated rows and add their total values

In [203]:
#dfVenues_grouped = dfVenues_onehot.groupby('Neighborhood').mean().reset_index()
dfVenues_grouped = dfVenues_onehot.groupby('Neighborhood', axis=0).sum()
dfVenues_grouped

,Hotel,Rental Car Location
Neighborhood,,
Aeropuerto,1,1
Alameda de Osuna,1,1
Almenara,0,1
Apostol Santiago,0,1
Aravaca,0,2
Atalaya,0,1
Bellas Vistas,0,1
Berruguete,0,1
Canillas,0,1


In [204]:
print('At this moment, we have ', dfVenues_grouped.shape[0], ' candidate neighborhoods')

At this moment, we have  43  candidate neighborhoods


### We classify the neighborhood candidates 

In [205]:
dfVenues_grouped['Points'] = dfVenues_grouped['Hotel'] + dfVenues_grouped['Rental Car Location']
dfVenues_grouped.head()

,Hotel,Rental Car Location,Points
Neighborhood,,,
Aeropuerto,1,1,2
Alameda de Osuna,1,1,2
Almenara,0,1,1
Apostol Santiago,0,1,1
Aravaca,0,2,2


In [206]:
print('Here is our clasification according to our point system:')
dfVenues_sorted = dfVenues_grouped.sort_values(by='Points', ascending=False, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)
dfVenues_sorted

Here is our clasification according to our point system:


,Hotel,Rental Car Location,Points
Neighborhood,,,
La Chopera,1,2,3
Delicias,1,2,3
Los Rosales,1,2,3
Comillas,1,2,3
Santa Eugenia,1,1,2
Rosas,1,1,2
Rejas,1,1,2
Palos de Moguer,0,2,2
Palomas,1,1,2


In [207]:
dfCandidates = dfVenues_sorted[dfVenues_sorted['Points'] > 2 ].index
print('Here we have the Neighborhoods with the highest score:')
dfCandBarrios = pd.DataFrame(dfCandidates)
dfCandBarrios

Here we have the Neighborhoods with the highest score:


,Neighborhood
0,La Chopera
1,Delicias
2,Los Rosales
3,Comillas


We take their values in the full table

In [208]:
#neighborhoods_cand = neighborhoods[neighborhoods['Neighborhood']==dfCandBarrios['Neighborhood'].to_list()]
neighborhoods_cand = neighborhoods[neighborhoods['Neighborhood'].isin(dfCandBarrios['Neighborhood'])]
neighborhoods_cand.head()

,District,Neighborhood,Latitude,Longitude,Surface (km2),Density (hab/km2),Price (€/m2)
8,Arganzuela,La Chopera,-3.699556,40.394639,0.56,35276.785714,3625.0
10,Arganzuela,Delicias,-3.693955,40.398093,1.07,25485.046729,3904.0
65,Carabanchel,Comillas,-3.711391,40.392854,0.67,33000.000000,2702.0
109,Villaverde,Los Rosales,-3.688816,40.354694,1.51,24163.576159,2869.0


In [209]:
map_madrid3 = folium.Map(location=[longitude_sol, latitude_sol], zoom_start=12)

# add markers to map
for lng, lat, district, neighborhood in zip(neighborhoods_cand['Latitude'], neighborhoods_cand['Longitude'], neighborhoods_cand['District'], neighborhoods_cand['Neighborhood']):
    label = '{}, {}'.format(district,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid3)  
    
map_madrid3